# References
KoGPT3 shares the same structure as KoGPT2. 

- [KoGPT2-Transformers huggingface 활용 예시](https://github.com/taeminlee/KoGPT2-Transformers)

In [2]:
from transformers import GPT2Tokenizer, PreTrainedTokenizerFast

model_dir = "skt/ko-gpt-trinity-1.2B-v0.5"

# Load the Tokenizer: "Fast" means that the tokenizer code is written in Rust Lang
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    model_dir,
    bos_token="<s>",
    eos_token="</s>",
    unk_token="<unk>",
    pad_token="<pad>",
    mask_token="<mask>",
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [3]:
from transformers import GPT2LMHeadModel

# designate the model's name registered on huggingface: https://huggingface.co/skt/ko-gpt-trinity-1.2B-v0.5
model_dir = "skt/ko-gpt-trinity-1.2B-v0.5"

# Attach Language model Head to the pretrained GPT model
model = GPT2LMHeadModel.from_pretrained(model_dir) # KoGPT3 shares the same structure as KoGPT2. 

In [4]:
# move the model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 1920)
    (wpe): Embedding(1024, 1920)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1920,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
     

In [5]:
# encode the sample sentence
sample = "이 편지는 영국에서 최초로 시작되어 일년에 한바퀴 돌면서 받는 사람에게 행운을 주었고 지금은 당신에게로 옮겨진 이 편지는"
print(tokenizer.encode(sample))

[29976, 30296, 30248, 50056, 33792, 30300, 30318, 30002, 37783, 29991, 44631, 30247, 30083, 31755, 35027, 41144, 25772, 30003, 34224, 32886, 47133, 21956, 34040, 26412, 29976, 30296, 30248]


In [7]:
import torch

torch.manual_seed(42)

# encode the sample sentence
input_ids = tokenizer.encode(sample, add_special_tokens=False, return_tensors="pt")

# generate output sequence from the given encoded input sequence
output_sequences = model.generate(input_ids=input_ids, do_sample=True, max_length=150, num_return_sequences=3)

# decode the output sequence and print its outcome
for index, generated_sequence in enumerate(output_sequences):
    generated_sequence = generated_sequence.tolist()
    decoded_sequence = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)
    print(f"Generated Sequence | Number {index} : {decoded_sequence}")
    print()


Generated Sequence | Number 0 : 이 편지는 영국에서 최초로 시작되어 일년에 한바퀴 돌면서 받는 사람에게 행운을 주었고 지금은 당신에게로 옮겨진 이 편지는 이 지구 상의 모든 사람에게 복을 주는 것이 된다. 이 편지는 그 내용이 진실되고 당신이 다른 사람에게 복을 주기를 바라는 마음이 담겨져 있다. 그리고 당신의 친구들은 이 편지를 읽고 당신의 행운을 기원해주며 당신에게 다시 연락한다. 마지막으로 당신의 친구들은 이 편지를 읽을 때마다 복을 받길 기원하며 당신에게 행운을 가져다 줄 것이다. 이 우체통에 넣은 편지는 당신이 한밤중에 받아도 좋을 것이다. 그 편지 안에는 당신이 받을 것으로 예상하는 것에 대한 목록이 나와 있다. 그리고 당신이 생각하고 있는 행운에 대한 목록이 있다. 당신에게 좋은 일이나 불행한 일을 예상해보기 바란다.
 행운 편지지와 함께 당신이 예상

Generated Sequence | Number 1 : 이 편지는 영국에서 최초로 시작되어 일년에 한바퀴 돌면서 받는 사람에게 행운을 주었고 지금은 당신에게로 옮겨진 이 편지는 지금 당신의 가장 가까운 사람에게 당신의 가장 진실한 사랑을 전하는 편지입니다. 
 
 (
 내가 가장 아끼는 사람은... ) 
 <unk> 당신이 나의 삶의 가장 큰 행복이랍니다. 
 <unk> 당신이 나의 가장 사랑하는 사람이랍니다. 
 <unk> 나는 당신에게 한평생 좋은 친구가 될 것입니다. 
 <unk> 당신은 나와 우정을 나누면 좋은 친구이고 
 <unk> 나는 당신에게 일생의 동반자가 될 것입니다. 
 <unk> 당신은 나와 행복할 것이고 나와 슬픔을 나누면 행복할 것입니다. 
 <unk> 당신은 나와 하나가 될 것이고 당신이 바로 나입니다. 
 <unk> 당신은 나의 가장 소중한 친구입니다. 
 <unk> 당신은 나의

Generated Sequence | Number 2 : 이 편지는 영국에서 최초로 시작되어 일년에 한바퀴 돌면서 받는 사람에게 행운을 주었고 지금은 당신에게로 옮겨진